In [264]:
import numpy as np
from sklearn.linear_model import RidgeCV, LassoCV, HuberRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.neural_network import MLPRegressor
import random
from sklearn.linear_model import Ridge, RidgeCV, LassoCV

In [265]:
# read traning data
df = pd.read_csv("training_data.csv",low_memory=False)

In [266]:
vehicle_make_description=df['Vehicle_Make_Description'].values
short_name=[]
for i in range(len(vehicle_make_description)):
    name=vehicle_make_description[i][:3]
    short_name.append(name)
vehicles_short_name = pd.DataFrame(data = short_name)
df['Vehicle_Make_Description'] = vehicles_short_name

In [267]:
df=df.dropna()



In [268]:
for col in list(df):
    print(col)
    print(df[col].unique())

PolicyNo
[164532941 164533241 164534633 ... 381735600 382057400 382162500]
Policy_Company
['Standard ' 'Preferred']
Policy_Installment_Term
[ 6 12]
Policy_Billing_Code
['Direct Billed to Insured' 'Premium Finance']
Policy_Method_Of_Payment
['Pre-paid' 'Installment']
Policy_Reinstatement_Fee_Indicator
['N' 'Y']
Policy_Zip_Code_Garaging_Location
['43046' 'Unknown' '43555' ... '52183' '14344' '43191']
Vehicle_Territory
[35 17 27 30 31 32 26 13 16 36 37 20 18 22 15 19]
Vehicle_Make_Year
[2004 1980 2005 1985 1991 2001 1995 1997 1998 1990 1986 1992 1996 1984
 2002 1988 2006 1999 2003 2000 1975 1993 1982 1994 2007 1978 1983 1987
 1977 1973 1989 1969 1976 1954 1979 1970 1955 1964 1981 1966 1972 1965
 1971 1961 1962 1967 1968 1974 1960 1952 1953 1940 1963 1959 1957 1950
 1928 1956 1947 1948 1951 1939 1958 2008 1949 1941 1934 1930 1932 1937
 1924 1935 1938 1931 1936 1946 1945 1929 1942 1927]
Vehicle_Make_Description
['BUI' 'CAD' 'PON' 'MER' 'GMC' 'FOR' 'CHE' 'NSS' 'DOD' 'LIN' 'MAZ' 'HON'
 'JAG' 

In [269]:
print("Unknown values:")
for col in list(df):
    a=df[(df[col] =="Unknown" )]
    if(len(a)!=0):
        print(col)
        print(len(a))
print("-1  values")
for col in list(df):
    a=df[(df[col] ==-1 )]
    if(len(a)!=0):
        print(col)
        print(len(a))
print("1M - 1M values:")
for col in list(df):
    a=df[(df[col] =='1M-1M  ')]
    if(len(a)!=0):
        print(col)
        print(len(a))
print("99 values:")
for col in list(df):
    a=df[(df[col] ==99)]
    if(len(a)!=0):
        print(col)
        print(len(a))

Unknown values:


C:\Users\dell\Anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


Policy_Zip_Code_Garaging_Location
3402
Vehicle_Annual_Miles
407018
Vehicle_Anti_Theft_Device
3
Vehicle_Passive_Restraint
78
EEA_Policy_Zip_Code_3
3402
-1  values
Vehicle_New_Cost_Amount
400355
Vehicle_Symbol
25
Vehicle_Miles_To_Work
253930
Vehicle_Days_Per_Week_Driven
18
Vehicle_Med_Pay_Limit
93798
Vehicle_Physical_Damage_Limit
12
Vehicle_Comprehensive_Coverage_Limit
407105
Vehicle_Collision_Coverage_Deductible
160145
EEA_Policy_Tenure
40
1M - 1M values:
Vehicle_Bodily_Injury_Limit
42
99 values:
Vehicle_Number_Of_Drivers_Assigned
114430
Vehicle_Miles_To_Work
64
Driver_Minimum_Age
12
Driver_Maximum_Age
14
Annual_Premium
3


In [270]:
#Observing which columns have same value or most missing as they won't affect the target variable and then dropping them
dropping_columns=['PolicyNo','EEA_PolicyYear','Policy_Zip_Code_Garaging_Location','Vehicle_Annual_Miles','Vehicle_New_Cost_Amount','Vehicle_Comprehensive_Coverage_Limit']
df = df.drop(dropping_columns, axis=1)


In [271]:

df=df[(df['Vehicle_Anti_Theft_Device'] !="Unknown" )]
df=df[(df['Vehicle_Passive_Restraint'] !="Unknown" )]
df=df[(df['EEA_Policy_Zip_Code_3'] !="Unknown" )]
df=df[(df['Vehicle_Symbol'] !=-1 )]
df=df[(df['Vehicle_Bodily_Injury_Limit'] !='1M-1M  ' )]
df=df[(df['Driver_Maximum_Age']!=99)]
df=df[(df['Driver_Minimum_Age']!=99)]
df=df[(df.Driver_Maximum_Age != 0 )]
df=df[(df.Driver_Minimum_Age != 0 )]
df.shape

(403546, 63)

In [272]:
#-1 values Vehicle_Days_Per_Week_Driven,
df['Vehicle_Days_Per_Week_Driven'].replace(-1,np.NaN,inplace=True)
Vehicle_Days_Per_Week_Driven_Mode=df['Vehicle_Days_Per_Week_Driven'].mode()
df['Vehicle_Days_Per_Week_Driven']=df['Vehicle_Days_Per_Week_Driven'].fillna(float(Vehicle_Days_Per_Week_Driven_Mode))
df['Vehicle_Days_Per_Week_Driven'].unique()

array([5., 4., 6., 3., 1., 0., 2., 7., 8., 9.])

In [273]:
df['Vehicle_Miles_To_Work'].replace(-1,np.NaN,inplace=True)
Vehicle_Miles_To_Work_Mean=df['Vehicle_Miles_To_Work'].mean()
a=int(Vehicle_Miles_To_Work_Mean)
df['Vehicle_Miles_To_Work']=df['Vehicle_Miles_To_Work'].fillna(a)
df['Vehicle_Miles_To_Work'].unique()

array([10., 12.,  7., 21., 14.,  8.,  4.,  2., 13.,  6.,  5., 25., 15.,
        3.,  1., 42., 20., 35., 40., 30., 50., 11.,  9., 22., 16., 26.,
       17., 23., 18.,  0., 24., 27., 60., 31., 29., 32., 99., 45., 28.,
       38., 49., 19., 37., 48., 34., 36., 95., 39., 62., 43., 52., 46.,
       65., 90., 55., 80., 58., 47., 44., 51., 57., 70., 71., 56., 33.,
       83., 75., 61., 64., 41., 93., 54., 63., 53., 78., 67., 72., 98.,
       59.])

In [274]:
# # Vehicle_Days_Per_Week_Driven
df['Vehicle_Days_Per_Week_Driven'].replace(-1,np.NaN,inplace=True)
Vehicle_Days_Per_Week_Driven_Mode=df['Vehicle_Days_Per_Week_Driven'].mode()
df['Vehicle_Days_Per_Week_Driven']=df['Vehicle_Days_Per_Week_Driven'].fillna(float(Vehicle_Days_Per_Week_Driven_Mode))
df['Vehicle_Days_Per_Week_Driven'].unique()

array([5., 4., 6., 3., 1., 0., 2., 7., 8., 9.])

In [275]:
# Vehicle_Med_Pay_Limit
df['Vehicle_Med_Pay_Limit'].replace(-1,np.NaN,inplace=True)
Vehicle_Med_Pay_Limit=df['Vehicle_Med_Pay_Limit'].mean()
df['Vehicle_Med_Pay_Limit']=df['Vehicle_Med_Pay_Limit'].fillna(int(Vehicle_Med_Pay_Limit))
df['Vehicle_Med_Pay_Limit'].unique()

array([ 1000., 50000.,  5000.,  3476., 10000., 25000.,  2000.])

In [276]:
# Vehicle_Physical_Damage_Limit
df['Vehicle_Physical_Damage_Limit'].replace(-1,np.NaN,inplace=True)
Vehicle_Physical_Damage_Limit_Mean=df['Vehicle_Physical_Damage_Limit'].mean()
df['Vehicle_Physical_Damage_Limit']=df['Vehicle_Physical_Damage_Limit'].fillna(int(Vehicle_Physical_Damage_Limit_Mean))
df['Vehicle_Physical_Damage_Limit'].unique()

array([100000.,  35000.,  25000.,  50000., 250000., 500000.,  50033.])

In [277]:
# Vehicle_Number_Of_Drivers_Assigned
df['Vehicle_Number_Of_Drivers_Assigned'].replace(99,np.NaN,inplace=True)
Vehicle_Number_Of_Drivers_Assigned_Mode=df['Vehicle_Number_Of_Drivers_Assigned'].mean()
df['Vehicle_Number_Of_Drivers_Assigned']=df['Vehicle_Number_Of_Drivers_Assigned'].fillna(int(Vehicle_Number_Of_Drivers_Assigned_Mode))
df['Vehicle_Number_Of_Drivers_Assigned'].unique()

array([1., 2., 3., 4., 5., 8., 7., 6.])

In [278]:
# Vehicle_Collision_Coverage_Deductible
df['Vehicle_Collision_Coverage_Deductible'].replace(-1,np.NaN,inplace=True)
Vehicle_Collision_Coverage_Deductible_Mean=df['Vehicle_Collision_Coverage_Deductible'].mean()
df['Vehicle_Collision_Coverage_Deductible']=df['Vehicle_Collision_Coverage_Deductible'].fillna(int(Vehicle_Collision_Coverage_Deductible_Mean))
df['Vehicle_Collision_Coverage_Deductible'].unique()

array([ 500.,  100.,  250.,  485., 1000.,  200.,   50., 3000., 2000.])

In [279]:
# EEA_Policy_Tenure
df['EEA_Policy_Tenure'].replace(-1,np.NaN,inplace=True)
EEA_Policy_Tenure_Mean=df['EEA_Policy_Tenure'].mean()
df['EEA_Policy_Tenure']=df['EEA_Policy_Tenure'].fillna(round(EEA_Policy_Tenure_Mean,1))
df['EEA_Policy_Tenure'].unique()


array([22.7, 47.2, 46.7, 16.5, 46.3, 17. , 47.7, 16. , 46.6, 47.1, 16.1,
       16.6, 47.5, 16.8, 16.2, 16.7, 25.3, 16.3, 37.5, 42. , 42.5, 16.9,
       46. , 39.7, 47. , 47.9, 47.6, 16.4, 40.1, 38. , 47.3, 46.5, 43.2,
       13.8, 23.1, 23.6, 15.9, 46.9, 23. , 23.5, 18.3, 45.1, 44. , 29.8,
       45.6, 46.1, 30. , 45.9, 22.4, 44.5, 45.5, 45.7, 46.2, 15.8, 46.8,
       41. , 47.4, 24. , 45. , 45.3, 40.4, 40.9, 42.1, 44.6, 20.8, 20.5,
       20. , 43.5, 44.3, 39. , 35.3, 35.8, 43.6, 44.1, 30.2, 30.7, 43.9,
       40. , 40.5, 41.5, 43.1,  5.7, 25.1, 25.6, 43. , 42.7, 15.7, 17.3,
       21.3, 18.1, 18.6, 42.8, 20.1, 20.6, 15.6, 42.3, 42.6, 42.2, 30.5,
       41.9, 41.7, 15.5, 41.3, 42.4, 42.9, 17.5, 34. , 41.1, 38.4, 38.9,
       41.2, 41.8, 23.8, 41.6, 35. , 22.1, 41.4, 35.9, 39.1, 15.4, 40.2,
       40.6, 40.7, 28.8, 40.3, 40.8, 35.5, 30.8, 15.3, 37.8, 38.2, 35.6,
       29. , 20.2, 20.7, 27.4, 21.1, 39.3, 39.8, 39.5, 22.6, 39.6, 18.4,
       39.2, 38.5, 24.8, 21. , 19.8, 24.3, 27. , 20

In [280]:
#Vehicle_Number_Of_Drivers_Assigned 99 drivers
df['Vehicle_Number_Of_Drivers_Assigned'].replace(99,np.NaN,inplace=True)
# # Vehicle_Number_Of_Drivers_Assigned_Mode=df['Vehicle_Number_Of_Drivers_Assigned'].mode()
df['Vehicle_Number_Of_Drivers_Assigned']=df['Vehicle_Number_Of_Drivers_Assigned'].fillna(9)
df['Vehicle_Number_Of_Drivers_Assigned'].unique()

array([1., 2., 3., 4., 5., 8., 7., 6.])

In [281]:
target=df['Loss_Amount']
df = df.drop(df.columns[58:], axis=1)
df

,Policy_Company,Policy_Installment_Term,Policy_Billing_Code,Policy_Method_Of_Payment,Policy_Reinstatement_Fee_Indicator,Vehicle_Territory,Vehicle_Make_Year,Vehicle_Make_Description,Vehicle_Performance,Vehicle_Symbol,...,EEA_Multi_Auto_Policies_Indicator,EEA_Policy_Zip_Code_3,EEA_Policy_Tenure,EEA_Agency_Type,EEA_Packaged_Policy_Indicator,EEA_Full_Coverage_Indicator,EEA_Prior_Bodily_Injury_Limit,SYS_Renewed,SYS_New_Business,Annual_Premium
0,Standard,6,Direct Billed to Insured,Pre-paid,N,35,2004,BUI,Standard,11,...,N,430,22.7,Standard,N,Y,100-400,Y,N,320.12
2,Standard,6,Direct Billed to Insured,Pre-paid,N,17,2005,PON,Standard,10,...,N,435,47.2,Non-standard,N,Y,100-400,Y,N,613.74
3,Standard,6,Direct Billed to Insured,Pre-paid,N,17,2005,MER,Standard,12,...,Y,435,46.7,Non-standard,Y,Y,40-100,Y,N,541.66
4,Standard,6,Direct Billed to Insured,Pre-paid,N,17,2005,MER,Standard,12,...,Y,435,47.2,Non-standard,Y,Y,40-100,Y,N,541.66
5,Standard,6,Direct Billed to Insured,Pre-paid,N,27,1985,GMC,Standard,7,...,Y,430,16.5,Hybrid,Y,N,20-50,Y,N,164.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424424,Standard,6,Direct Billed to Insured,Pre-paid,N,31,2001,DOD,Standard,16,...,Y,434,0.0,Non-standard,Y,Y,20-50,N,Y,649.55
424425,Standard,12,Premium Finance,Pre-paid,N,36,2007,NSS,Standard,7,...,Y,425,0.0,Non-standard,Y,Y,100-200,Y,Y,541.26
424427,Standard,6,Direct Billed to Insured,Pre-paid,N,31,2000,NSS,Standard,8,...,Y,436,0.0,Hybrid,Y,N,100-200,Y,Y,117.13
424428,Standard,6,Direct Billed to Insured,Installment,N,35,1997,PON,Standard,8,...,Y,424,0.0,Preferred,N,Y,100-400,N,Y,118.21


In [282]:
le = LabelEncoder()
for col in df.columns:
    col = str(col)
    if str(df.loc[:,col].dtype) == 'object':
        le.fit(df.loc[:,col])
        df.loc[:,col] = le.transform(df.loc[:,col])
df

,Policy_Company,Policy_Installment_Term,Policy_Billing_Code,Policy_Method_Of_Payment,Policy_Reinstatement_Fee_Indicator,Vehicle_Territory,Vehicle_Make_Year,Vehicle_Make_Description,Vehicle_Performance,Vehicle_Symbol,...,EEA_Multi_Auto_Policies_Indicator,EEA_Policy_Zip_Code_3,EEA_Policy_Tenure,EEA_Agency_Type,EEA_Packaged_Policy_Indicator,EEA_Full_Coverage_Indicator,EEA_Prior_Bodily_Injury_Limit,SYS_Renewed,SYS_New_Business,Annual_Premium
0,1,6,0,1,0,35,2004,25,4,11,...,0,68,22.7,3,0,1,1,1,0,320.12
2,1,6,0,1,0,17,2005,145,4,10,...,0,73,47.2,1,0,1,1,1,0,613.74
3,1,6,0,1,0,17,2005,123,4,12,...,1,73,46.7,1,1,1,5,1,0,541.66
4,1,6,0,1,0,17,2005,123,4,12,...,1,73,47.2,1,1,1,5,1,0,541.66
5,1,6,0,1,0,27,1985,84,4,7,...,1,68,16.5,0,1,0,2,1,0,164.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424424,1,6,0,1,0,31,2001,56,4,16,...,1,72,0.0,1,1,1,2,0,1,649.55
424425,1,12,1,1,0,36,2007,137,4,7,...,1,63,0.0,1,1,1,0,1,1,541.26
424427,1,6,0,1,0,31,2000,137,4,8,...,1,74,0.0,0,1,0,0,1,1,117.13
424428,1,6,0,0,0,35,1997,145,4,8,...,1,62,0.0,2,0,1,1,0,1,118.21


In [283]:
#df.dropna(subset = ["Annual_Premium"], inplace=True)
df.isnull().sum()

Policy_Company                              0
Policy_Installment_Term                     0
Policy_Billing_Code                         0
Policy_Method_Of_Payment                    0
Policy_Reinstatement_Fee_Indicator          0
Vehicle_Territory                           0
Vehicle_Make_Year                           0
Vehicle_Make_Description                    0
Vehicle_Performance                         0
Vehicle_Symbol                              0
Vehicle_Number_Of_Drivers_Assigned          0
Vehicle_Usage                               0
Vehicle_Miles_To_Work                       0
Vehicle_Days_Per_Week_Driven                0
Vehicle_Anti_Theft_Device                   0
Vehicle_Passive_Restraint                   0
Vehicle_Age_In_Years                        0
Vehicle_Med_Pay_Limit                       0
Vehicle_Bodily_Injury_Limit                 0
Vehicle_Physical_Damage_Limit               0
Vehicle_Comprehensive_Coverage_Indicator    0
Vehicle_Collision_Coverage_Indicat

In [284]:
df_copy=df.copy()

In [285]:
scaler = StandardScaler()
scaler.fit(df)
df_scaled=scaler.transform(df)
df = pd.DataFrame(df_scaled)
df

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.260796,-0.194625,-0.148395,1.023013,-0.367893,0.943944,0.989109,-1.343881,0.211284,-0.048291,...,-2.738221,-0.193375,2.648257,1.355151,-0.935114,0.801011,-0.575652,0.359098,-0.336556,-0.068619
1,0.260796,-0.194625,-0.148395,1.023013,-0.367893,-3.331673,1.126497,1.407333,0.211284,-0.298662,...,-2.738221,0.238775,6.472210,-0.391952,-0.935114,0.801011,-0.575652,0.359098,-0.336556,1.361103
2,0.260796,-0.194625,-0.148395,1.023013,-0.367893,-3.331673,1.126497,0.902943,0.211284,0.202080,...,0.365201,0.238775,6.394171,-0.391952,1.069388,0.801011,1.491787,0.359098,-0.336556,1.010124
3,0.260796,-0.194625,-0.148395,1.023013,-0.367893,-3.331673,1.126497,0.902943,0.211284,0.202080,...,0.365201,0.238775,6.472210,-0.391952,1.069388,0.801011,1.491787,0.359098,-0.336556,1.010124
4,0.260796,-0.194625,-0.148395,1.023013,-0.367893,-0.956330,-1.621271,0.008799,0.211284,-1.049775,...,0.365201,-0.193375,1.680562,-1.265503,1.069388,-1.248422,-0.058792,0.359098,-0.336556,-0.827352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403541,0.260796,-0.194625,-0.148395,1.023013,-0.367893,-0.006193,0.576944,-0.633151,0.211284,1.203564,...,0.365201,0.152345,-0.894754,-0.391952,1.069388,0.801011,-0.058792,-2.784753,2.971269,1.535472
403542,0.260796,5.138085,6.738789,1.023013,-0.367893,1.181478,1.401274,1.223918,0.211284,-1.049775,...,0.365201,-0.625524,-0.894754,-0.391952,1.069388,0.801011,-1.092512,0.359098,2.971269,1.008177
403543,0.260796,-0.194625,-0.148395,1.023013,-0.367893,-0.006193,0.439555,1.223918,0.211284,-0.799404,...,0.365201,0.325205,-0.894754,-1.265503,1.069388,-1.248422,-1.092512,0.359098,2.971269,-1.057037
403544,0.260796,-0.194625,-0.148395,-0.977505,-0.367893,0.943944,0.027390,1.407333,0.211284,-0.799404,...,0.365201,-0.711954,-0.894754,0.481600,-0.935114,0.801011,-0.575652,-2.784753,2.971269,-1.051778


In [286]:
pca = PCA(n_components=0.95)
principalComponents = pca.fit_transform(df)
principalDf = pd.DataFrame(data = principalComponents)
principalDf
df=principalDf.copy()
# pca = PCA(n_components=30).fit(df)
# principal_cols = pca.transform(df)
# df = pd.DataFrame(data = principal_cols)
df

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,0.021898,-3.108841,-1.158837,3.575376,-0.119242,2.214428,0.245883,1.978247,0.066509,0.825660,...,-0.389119,0.160094,0.575307,0.420190,-0.442733,0.512671,-1.064126,0.018924,-0.133564,0.604236
1,0.320501,-4.117123,-0.465979,4.858325,1.174517,2.260767,-0.960302,2.073748,1.337536,0.571509,...,1.853228,-1.002054,-0.810820,0.162367,-1.661612,0.816198,-1.268621,0.149280,-0.330381,1.026156
2,0.470361,-3.803084,-0.249355,3.348441,1.006890,3.466639,-1.729723,-1.246472,1.739382,1.181169,...,0.876638,-0.496573,0.085722,0.704878,-1.770166,-0.362288,-1.414089,0.416599,-0.026711,0.729304
3,0.444713,-3.849718,-0.241609,3.397896,1.008245,3.496114,-1.731320,-1.236319,1.747814,1.184279,...,0.890229,-0.485252,0.078105,0.704393,-1.790415,-0.378654,-1.433733,0.437350,-0.033758,0.731282
4,-2.317018,-0.369123,0.105726,-0.055863,-1.587042,-0.069546,0.437637,-0.442588,1.487044,0.557323,...,0.883872,0.333330,-0.942780,0.313192,-0.733578,0.340560,-0.025459,-0.889488,-0.144678,-0.392232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403541,4.037835,7.936314,2.059732,3.552443,2.150707,1.053484,-0.871199,-0.275925,-1.185191,-0.424566,...,7.553223,5.042420,8.345463,-0.284756,0.141301,5.776788,3.679310,3.805774,2.261303,-1.155774
403542,3.477832,-0.191341,-1.500166,-1.165852,5.937793,-3.275899,5.325033,-0.929476,2.526843,-0.872131,...,0.288852,2.064717,-1.074103,-0.181073,-0.529292,-0.338577,-0.347229,0.344098,-0.642523,2.285422
403543,1.362794,7.392149,0.997956,2.863489,-1.097914,-0.510724,1.254395,0.562845,-1.121561,-0.345888,...,2.792292,1.249410,2.821577,-0.352999,-1.564456,1.657828,1.808380,0.856409,-0.163248,1.073511
403544,-0.443953,-0.894339,-0.754904,-0.048778,1.851267,-1.726023,0.265799,0.116633,-3.319558,-1.092810,...,1.386537,-0.543547,-1.816114,-0.788280,-0.213705,0.383494,-0.185550,0.629773,-1.177798,0.687591


In [287]:
df['Loss_Amount'] = target
df['Annual_Premium'] = df_copy['Annual_Premium']
df.dropna(subset = ["Annual_Premium"], inplace=True)
df

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,Loss_Amount,Annual_Premium
0,0.021898,-3.108841,-1.158837,3.575376,-0.119242,2.214428,0.245883,1.978247,0.066509,0.825660,...,0.575307,0.420190,-0.442733,0.512671,-1.064126,0.018924,-0.133564,0.604236,0.0,320.12
2,0.470361,-3.803084,-0.249355,3.348441,1.006890,3.466639,-1.729723,-1.246472,1.739382,1.181169,...,0.085722,0.704878,-1.770166,-0.362288,-1.414089,0.416599,-0.026711,0.729304,0.0,613.74
3,0.444713,-3.849718,-0.241609,3.397896,1.008245,3.496114,-1.731320,-1.236319,1.747814,1.184279,...,0.078105,0.704393,-1.790415,-0.378654,-1.433733,0.437350,-0.033758,0.731282,0.0,541.66
4,-2.317018,-0.369123,0.105726,-0.055863,-1.587042,-0.069546,0.437637,-0.442588,1.487044,0.557323,...,-0.942780,0.313192,-0.733578,0.340560,-0.025459,-0.889488,-0.144678,-0.392232,0.0,541.66
5,1.168756,-4.388907,4.728072,2.738344,1.974252,1.080151,-0.124029,0.788132,-1.383441,1.641015,...,-0.981159,-0.087622,0.148477,-1.612573,-0.966439,0.574619,0.301955,-0.108659,0.0,164.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403541,4.037835,7.936314,2.059732,3.552443,2.150707,1.053484,-0.871199,-0.275925,-1.185191,-0.424566,...,8.345463,-0.284756,0.141301,5.776788,3.679310,3.805774,2.261303,-1.155774,0.0,167.48
403542,3.477832,-0.191341,-1.500166,-1.165852,5.937793,-3.275899,5.325033,-0.929476,2.526843,-0.872131,...,-1.074103,-0.181073,-0.529292,-0.338577,-0.347229,0.344098,-0.642523,2.285422,0.0,234.26
403543,1.362794,7.392149,0.997956,2.863489,-1.097914,-0.510724,1.254395,0.562845,-1.121561,-0.345888,...,2.821577,-0.352999,-1.564456,1.657828,1.808380,0.856409,-0.163248,1.073511,0.0,222.60
403544,-0.443953,-0.894339,-0.754904,-0.048778,1.851267,-1.726023,0.265799,0.116633,-3.319558,-1.092810,...,-1.816114,-0.788280,-0.213705,0.383494,-0.185550,0.629773,-1.177798,0.687591,0.0,526.82


In [288]:
df_copy2=df.copy()

In [377]:
def gen_portfolio(df):
    df = df.append(df, ignore_index=True)
    with_claims = []
    no_claims = []
    for i, r in enumerate(df['Loss_Amount']):
        if r > 0.0:
            with_claims.append(i)
        else:
            no_claims.append(i)
            
    random.shuffle(with_claims)
    random.shuffle(no_claims)
    portfolios = []
    i = 0
    while True:        
        size = np.random.choice(range(1, 4), p=[0.4, 0.3, 0.3])
        if(size==1):
            size=1000
        if(size==2):
            size=3000
        if(size==3):
            size=5000
        loss_percent = np.random.choice(range(0, 5), p=[0.2]*5) * 5
        if loss_percent == 0:
            loss_percent = 2
                
        portfolio = []
        num_losses = int(loss_percent*(size/100))
#         print(i,'-',size, loss_percent, num_losses)
        i += 1
        for i in range(num_losses):
            try:
                portfolio.append(df.iloc[with_claims.pop()])
            except Exception as e:
#                 print('breaking,', e)
                break

        for i in range(size-num_losses):
            try:
                portfolio.append(df.iloc[no_claims.pop()])
            except Exception as e:
#                 print('breaking,', e)
                break        

        if len(with_claims) == 0 or len(no_claims) == 0:                
            break
            
        random.shuffle(portfolio)
        portfolios.append(pd.DataFrame(portfolio, columns=df.columns))
    return portfolios

In [378]:
portfolio=gen_portfolio(df)
len(portfolio)
for p in portfolio:
    print(np.log(p['Loss_Amount'].sum()/p['Annual_Premium'].sum()))

0.6802679927027353
0.8480624924373481
-0.031808849333263535
-1.1416455215103611
1.0157800205434593
0.8420263724457705
-0.16078351502278332
0.10363143244233317
0.7900910366872171
1.1691756829888196
-0.282951288350428
0.28607369835518476
-0.4851688106302461
-1.8227530206302753
0.4232761826260956
-0.3806725864685173
1.1398378492971153
0.47529570906702484
-0.08442816372395985
-1.2571451945651204
0.9114347225439587
0.8227217142323175
-0.8561100378625109
-1.4553382377656194
-0.2515777113063294
0.8370772302412824
0.5097436790743918
-0.21043781377558216
0.7764487930505786
0.4523599486911155
-0.9952070260851087
0.6640203399013043
-1.4559887199750712
0.7243120876344331
-0.41908369355188035
-1.3369685105506148
-0.31594204309366064
-0.38008818540683154
-1.096041410612543
-0.4990305121299945
1.3000750272810477
0.210925549405289
0.45049275915433673
0.7344498749033479
0.852970531581461
-1.6768327387422282
1.1391679702420778
0.7425268013790157
0.17067714617062
-0.8958971775701571
0.8445869452916314
1.

In [379]:
def median_summarizer(df_t):    
    values = df_t.median()
    n_df = pd.DataFrame([], columns=df_t.columns)
    return n_df.append(values, ignore_index=True)

In [380]:
def summarize_portfolio(portfolio,convert_to_lr):
    summarized_portfolios = []
    flag = True
    for p in portfolio: 
        portfolios = median_summarizer(p)
        if convert_to_lr:
            portfolios['lr'] = np.log(p['Loss_Amount'].sum()/p['Annual_Premium'].sum())
            portfolios = portfolios.drop(['Loss_Amount', 'Annual_Premium'], axis=1)
        if flag:
                cols = portfolios.columns
                flag = False
        summarized_portfolios.append(portfolios)
    portoflio_attributes = pd.DataFrame([], columns=cols)
    for p in summarized_portfolios:
        portoflio_attributes = portoflio_attributes.append(p, ignore_index=True)
    return portoflio_attributes

In [381]:
training_portoflio_attributes=summarize_portfolio(portfolio,True)

In [382]:
training_portoflio_attributes


,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,lr
0,0.252734,-0.543350,-0.540904,-0.074693,0.054544,0.104755,-0.273407,-0.180811,0.041973,0.061375,...,-0.037489,-0.042010,-0.011605,0.102580,-0.005716,-0.009120,-0.013327,0.012292,-0.000743,0.680268
1,0.306300,-0.507517,-0.435749,-0.124748,0.000219,-0.064447,-0.202195,-0.095837,-0.017990,0.133910,...,-0.026496,-0.014718,-0.008996,0.125080,0.001768,-0.004270,-0.035186,0.037576,-0.016662,0.848062
2,0.400679,-0.512083,-0.439804,-0.048467,0.029400,-0.002191,-0.240213,-0.117617,0.000222,0.087016,...,-0.026038,-0.011135,-0.000461,0.104230,-0.000963,-0.008098,-0.005394,0.020616,0.002509,-0.031809
3,0.378799,-0.507622,-0.402881,-0.055798,0.049842,-0.104554,-0.215006,-0.122105,-0.018288,0.079716,...,-0.038397,-0.009461,-0.007771,0.119589,-0.022007,0.005265,-0.010690,0.049641,0.028644,-1.141646
4,0.377842,-0.470421,-0.484165,-0.065719,0.041385,-0.093705,-0.179927,-0.086791,-0.022752,0.018508,...,-0.015893,-0.008131,0.018546,0.107892,0.001947,0.004537,-0.011968,0.038355,0.017438,1.015780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,0.373848,-0.497149,-0.437306,-0.120730,0.020217,-0.065538,-0.209713,-0.078103,0.057107,0.111395,...,-0.039902,-0.021186,-0.009270,0.086567,-0.012623,0.012202,-0.021669,-0.007920,0.010089,-1.286796
113,0.412271,-0.529164,-0.446984,-0.045809,0.074968,-0.091091,-0.227576,-0.088898,0.051851,0.080017,...,-0.033693,-0.018247,-0.002857,0.096511,-0.011982,-0.014111,-0.013376,0.009479,0.012030,-1.210808
114,0.352825,-0.484709,-0.444177,-0.083665,-0.005096,-0.072287,-0.201781,-0.064546,0.018193,0.112743,...,-0.024821,-0.003502,-0.006750,0.099396,0.012023,0.001277,-0.032504,0.013253,-0.014489,-0.952088
115,0.434783,-0.546203,-0.430895,-0.051306,0.068462,-0.095538,-0.260183,-0.142197,0.040863,0.143549,...,-0.029436,-0.018348,0.001420,0.111804,-0.024659,-0.029629,0.005314,0.012872,-0.007807,0.820748


In [383]:
import glob
path = '2020_Spring_testing_portfolios' # use your path
all_files = glob.glob(path + "/*.csv")
print(len(all_files))
portfolios = []
i = 0
for filename in all_files:
    i += 1
    testing_data = pd.read_csv(filename, index_col=None, header=0)
    testing_data = testing_data.drop(dropping_columns, axis=1)
    vehicles = [v[:3] for v in testing_data['Vehicle_Make_Description'].values]
    vehicles = pd.DataFrame(data = vehicles)
    testing_data['Vehicle_Make_Description'] = vehicles
    testing_data=testing_data.dropna()
    testing_data=testing_data[(testing_data['Vehicle_Anti_Theft_Device'] !="Unknown" )]
    testing_data=testing_data[(testing_data['Vehicle_Passive_Restraint'] !="Unknown" )]
    testing_data=testing_data[(testing_data['EEA_Policy_Zip_Code_3'] !="Unknown" )]
    testing_data=testing_data[(testing_data['Vehicle_Symbol'] !=-1 )]
    testing_data=testing_data[(testing_data['Vehicle_Bodily_Injury_Limit'] !='1M-1M  ' )]
    testing_data=testing_data[(testing_data['Driver_Maximum_Age']!=99)]
    testing_data=testing_data[(testing_data['Driver_Minimum_Age']!=99)]
    testing_data=testing_data[(testing_data.Driver_Maximum_Age != 0 )]
    testing_data=testing_data[(testing_data.Driver_Minimum_Age != 0 )]
    testing_data['Vehicle_Days_Per_Week_Driven'].replace(-1,np.NaN,inplace=True)
    Vehicle_Days_Per_Week_Driven_Mode=testing_data['Vehicle_Days_Per_Week_Driven'].mode()
    testing_data['Vehicle_Days_Per_Week_Driven']=testing_data['Vehicle_Days_Per_Week_Driven'].fillna(float(Vehicle_Days_Per_Week_Driven_Mode))
    testing_data['Vehicle_Miles_To_Work'].replace(-1,np.NaN,inplace=True)
    Vehicle_Miles_To_Work_Mean=testing_data['Vehicle_Miles_To_Work'].mean()
    a=int(Vehicle_Miles_To_Work_Mean)
    testing_data['Vehicle_Miles_To_Work']=testing_data['Vehicle_Miles_To_Work'].fillna(a)
    testing_data['Vehicle_Med_Pay_Limit'].replace(-1,np.NaN,inplace=True)
    Vehicle_Med_Pay_Limit=testing_data['Vehicle_Med_Pay_Limit'].mean()
    testing_data['Vehicle_Med_Pay_Limit']=testing_data['Vehicle_Med_Pay_Limit'].fillna(int(Vehicle_Med_Pay_Limit))
    testing_data['Vehicle_Physical_Damage_Limit'].replace(-1,np.NaN,inplace=True)
    Vehicle_Physical_Damage_Limit_Mean=testing_data['Vehicle_Physical_Damage_Limit'].mean()
    testing_data['Vehicle_Physical_Damage_Limit']=testing_data['Vehicle_Physical_Damage_Limit'].fillna(int(Vehicle_Physical_Damage_Limit_Mean))
    testing_data['Vehicle_Number_Of_Drivers_Assigned'].replace(99,np.NaN,inplace=True)
    Vehicle_Number_Of_Drivers_Assigned_Mode=testing_data['Vehicle_Number_Of_Drivers_Assigned'].mean()
    testing_data['Vehicle_Number_Of_Drivers_Assigned']=testing_data['Vehicle_Number_Of_Drivers_Assigned'].fillna(int(Vehicle_Number_Of_Drivers_Assigned_Mode))
    testing_data['Vehicle_Collision_Coverage_Deductible'].replace(-1,np.NaN,inplace=True)
    Vehicle_Collision_Coverage_Deductible_Mean=testing_data['Vehicle_Collision_Coverage_Deductible'].mean()
    testing_data['Vehicle_Collision_Coverage_Deductible']=testing_data['Vehicle_Collision_Coverage_Deductible'].fillna(int(Vehicle_Collision_Coverage_Deductible_Mean))
    testing_data['EEA_Policy_Tenure'].replace(-1,np.NaN,inplace=True)
    EEA_Policy_Tenure_Mean=testing_data['EEA_Policy_Tenure'].mean()
    testing_data['EEA_Policy_Tenure']=testing_data['EEA_Policy_Tenure'].fillna(round(EEA_Policy_Tenure_Mean,1))
    testing_data['Vehicle_Number_Of_Drivers_Assigned'].replace(99,np.NaN,inplace=True)
    testing_data['Vehicle_Number_Of_Drivers_Assigned']=testing_data['Vehicle_Number_Of_Drivers_Assigned'].fillna(9)
    le = LabelEncoder()
    for col in testing_data.columns:
        col = str(col)
        if str(testing_data.loc[:,col].dtype) == 'object':
            le.fit(testing_data.loc[:,col])
            testing_data.loc[:,col] = le.transform(testing_data.loc[:,col])
    testing_data_scaled = scaler.transform(testing_data)
    testing_data = pd.DataFrame(testing_data_scaled)
    testing_data_pca = pca.transform(testing_data)
    testing_data = pd.DataFrame(data = testing_data_pca)
    portfolios.append(testing_data)

330


In [384]:
testing_portfolios_attributes=summarize_portfolio(portfolios,False)

In [385]:
testing_portfolios_attributes


,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,0.415639,-0.609132,-0.461352,0.142086,-0.025785,-0.884946,-0.144793,-0.167756,0.195834,-0.008193,...,-1.450147,-2.159291,0.951390,0.340320,0.214171,0.147668,-0.234434,0.029309,0.158554,-0.052456
1,0.544122,-0.565094,-0.566886,0.223772,-0.013655,-0.884011,-0.180244,-0.215707,0.123299,0.003265,...,-1.394567,-2.136160,0.919090,0.292619,0.227671,0.165278,-0.243901,0.031324,0.124699,-0.081516
2,0.228740,-0.443253,-0.485343,0.141143,-0.051114,-0.957238,-0.148102,-0.171321,0.147225,0.002632,...,-1.359390,-2.068137,0.951879,0.375003,0.225463,0.129228,-0.242433,0.062143,0.234420,-0.046034
3,0.450934,-0.702176,-0.486118,0.385100,-0.069982,-0.966124,-0.171310,-0.054057,0.255454,-0.063114,...,-1.365336,-2.106270,0.934506,0.297672,0.225576,0.162776,-0.219248,0.022200,0.230332,-0.022083
4,0.465863,-0.545438,-0.518676,0.158260,0.103952,-0.811413,-0.204409,-0.240930,0.153531,0.006297,...,-1.365763,-2.098162,0.901284,0.324631,0.235998,0.158312,-0.165158,-0.001562,0.213216,-0.098076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,0.480554,-0.552611,-0.435762,0.292904,0.013241,-1.018607,-0.143310,-0.111292,0.046120,-0.034243,...,-1.397184,-2.133352,0.925788,0.321775,0.207607,0.154697,-0.159010,0.048708,0.147742,-0.135174
326,0.550142,-0.509718,-0.557635,0.042833,0.023368,-0.883218,-0.165734,-0.292894,0.081475,-0.089096,...,-1.389748,-2.101781,0.899606,0.308566,0.247569,0.201584,-0.186869,0.007738,0.080053,-0.098891
327,0.387228,-0.396196,-0.468440,0.095245,0.044579,-0.678603,-0.165395,-0.258088,0.135400,-0.010355,...,-1.427018,-2.095825,0.947391,0.292796,0.249825,0.148993,-0.269306,0.042274,0.181925,-0.051866
328,0.430806,-0.454079,-0.470059,0.032911,-0.089594,-0.735164,-0.117712,-0.311770,0.042219,0.001096,...,-1.412154,-2.085050,0.924200,0.319873,0.214091,0.120505,-0.224619,0.023955,0.171362,-0.130153


In [386]:
def getOutput(predictions,filename):
    path = "2020_Spring_testing_portfolios" # use your path
    output_file=filename
    all_files = glob.glob(path + "/*.csv")
    print(f'found {len(all_files)} files.')
    files = []
    for f in all_files:
        t = f[36:]
        t = t[:-4]
        files.append(t)
    def sort_file(item):
        return int(item.split('_')[1])
    files = sorted(files, key=sort_file)
    assert len(predictions) == len(files)
    output = []
    print(f'constructing output for {len(predictions)} predictions')
    for i in range(len(predictions)):
        pred = predictions[i]
        output.append((files[i], pred))

    if output_file is not None:
        op_pd = pd.DataFrame(output, columns=['ID','ln_LR'])
        op_pd.to_csv(output_file + '.csv', index=False)

In [387]:
training=training_portoflio_attributes.copy()
loss_ratio=training['lr']
training=training.drop('lr',axis=1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(training.values,loss_ratio.values, test_size=0.25, random_state=42)


array([[ 0.44459611, -0.51096054, -0.46509512, ..., -0.0215613 ,
        -0.01092999,  0.00146795],
       [ 0.37784177, -0.47042118, -0.48416455, ..., -0.01196756,
         0.0383551 ,  0.01743811],
       [ 0.50595972, -0.47532573, -0.47505909, ..., -0.01585127,
         0.00913038,  0.01633289],
       ...,
       [ 0.44445372, -0.49518354, -0.44992091, ..., -0.01995229,
         0.04211383, -0.0066832 ],
       [ 0.44072792, -0.48501966, -0.43925832, ..., -0.03261311,
         0.01996592, -0.00164807],
       [ 0.36733908, -0.50119094, -0.44964203, ..., -0.00408531,
         0.02433096,  0.00307419]])

In [396]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
gsc = GridSearchCV( estimator=RandomForestRegressor(),
        param_grid={
            'max_depth': range(3,7),
            'n_estimators': (500, 550),
        },cv=5, scoring='neg_mean_squared_error', verbose=0,n_jobs=-1)
    
grid_result = gsc.fit(X_train, y_train)
best_params = grid_result.best_params_

rfr = RandomForestRegressor(max_depth=best_params["max_depth"], n_estimators=best_params["n_estimators"], random_state=False, verbose=False)



C:\Users\dell\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [397]:
rfr.fit(X_train,y_train)
predictions = rfr.predict(X_test)
errors = abs(predictions - y_test)
mape = 100 * np.mean(errors / y_test)
accuracy = 100 - mape
print('Model Performance')
print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
print('Accuracy = {:0.2f}%.'.format(accuracy))


Model Performance
Average Error: 0.6545 degrees.
Accuracy = 79.33%.


In [393]:
pred=rfr.predict(testing_portfolios_attributes)
pred

array([-0.22145397, -0.21223025, -0.25099824, -0.21304087, -0.24778936,
       -0.28952572, -0.23590949, -0.20095489, -0.10849375, -0.19660635,
       -0.24275137, -0.23171353, -0.19035065, -0.23948903, -0.30403   ,
       -0.26461792, -0.13181292, -0.23427475, -0.19233181, -0.1827304 ,
       -0.10390552, -0.2754702 , -0.19697129, -0.20925978, -0.27811791,
       -0.20635488, -0.22551067, -0.22457396, -0.12687014, -0.20372294,
       -0.25228335, -0.20195968, -0.23454475, -0.22903863, -0.23957756,
       -0.2053242 , -0.2232535 , -0.23576003, -0.16930258, -0.24656942,
       -0.26951708, -0.11218845, -0.21256489, -0.07771048, -0.28268858,
       -0.17772754, -0.23914603, -0.27911753, -0.21415714, -0.22800135,
       -0.19260408, -0.203782  , -0.26330046, -0.21898958, -0.27002587,
       -0.21152215, -0.16898232, -0.13188632, -0.20007581, -0.20769244,
       -0.26314897, -0.25957441, -0.2857286 , -0.21677598, -0.22076816,
       -0.22575311, -0.21793979, -0.26369873, -0.22779662, -0.19

In [312]:

getOutput(pred,"RFR")

found 330 files.
constructing output for 330 predictions


In [193]:
l2_cv = RidgeCV(cv=None, store_cv_values=True, alphas=np.linspace(0.1,1000,100))
l2_cv.fit(X_train, y_train)

RidgeCV(alphas=array([1.000e-01, 1.020e+01, 2.030e+01, 3.040e+01, 4.050e+01, 5.060e+01,
       6.070e+01, 7.080e+01, 8.090e+01, 9.100e+01, 1.011e+02, 1.112e+02,
       1.213e+02, 1.314e+02, 1.415e+02, 1.516e+02, 1.617e+02, 1.718e+02,
       1.819e+02, 1.920e+02, 2.021e+02, 2.122e+02, 2.223e+02, 2.324e+02,
       2.425e+02, 2.526e+02, 2.627e+02, 2.728e+02, 2.829e+02, 2.930e+02,
       3.031e+02, 3.132e+02, 3.233e+0...
       7.273e+02, 7.374e+02, 7.475e+02, 7.576e+02, 7.677e+02, 7.778e+02,
       7.879e+02, 7.980e+02, 8.081e+02, 8.182e+02, 8.283e+02, 8.384e+02,
       8.485e+02, 8.586e+02, 8.687e+02, 8.788e+02, 8.889e+02, 8.990e+02,
       9.091e+02, 9.192e+02, 9.293e+02, 9.394e+02, 9.495e+02, 9.596e+02,
       9.697e+02, 9.798e+02, 9.899e+02, 1.000e+03]),
        cv=None, fit_intercept=True, gcv_mode=None, normalize=False,
        scoring=None, store_cv_values=True)

In [194]:
l2_cv_pred=l2_cv.predict(testing_portfolios_attributes)

In [195]:
l2_cv_pred

array([-0.5425336 , -0.54323993, -0.54227902, -0.54228157, -0.54264124,
       -0.54224162, -0.54291478, -0.54411299, -0.54303269, -0.54318764,
       -0.54372272, -0.5425323 , -0.54174653, -0.54260893, -0.54260661,
       -0.54237307, -0.54367206, -0.54309064, -0.54266348, -0.5434486 ,
       -0.54241986, -0.54238349, -0.5436184 , -0.54246025, -0.54322943,
       -0.54336363, -0.54304285, -0.54199889, -0.54113197, -0.54282938,
       -0.54313399, -0.54319366, -0.54256826, -0.54252505, -0.54244343,
       -0.54421037, -0.54229163, -0.54241092, -0.54172509, -0.54223226,
       -0.54153291, -0.54282917, -0.54309117, -0.54174178, -0.54223932,
       -0.54208413, -0.54220162, -0.54234937, -0.54321516, -0.54325026,
       -0.54342634, -0.54389622, -0.54364699, -0.54246262, -0.54416703,
       -0.54418737, -0.54415125, -0.54263573, -0.54380014, -0.54323047,
       -0.54197021, -0.54297526, -0.54275722, -0.54214781, -0.54286182,
       -0.54244806, -0.5424152 , -0.54263301, -0.54284588, -0.54